In [20]:
import pandas as pd
import json
import urllib.parse
import base64
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from transformers import BertTokenizer, BertModel
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from tqdm import tqdm
from gensim.models import Word2Vec

df = pd.read_csv("Datasets/SQL_Dataset.csv", encoding='latin-1').sample(10000).reset_index(drop=True)
df

,Query,Label
0,SELECT * FROM universe WHERE fifty = 'better' ...,0
1,SELECT AVG ( source ) FROM failed SELECT SUM (...,0
2,1' ) ) ) waitfor delay '0:0:5' and ...,1
3,"INSERT INTO seldom ( tomorrow, nearly, scient...",0
4,prado de la guzpea,2
...,...,...
9995,SELECT * FROM highway WHERE NOT exact = 'pe...,0
9996,retasac5n,2
9997,"1"" ) ) ) waitfor delay '0:0:5'--",1
9998,SELECT * FROM bend WHERE own NOT LIKE '[needl...,0


In [21]:
# Определение функции декодирования
def decode_sql(encoded_string):
    # Декодирование в ASCII
    try:
        decoded_string = bytes.fromhex(encoded_string).decode('ascii')
    except:
        pass

    # Декодирование в Unicode
    try:
        decoded_string = bytes.fromhex(encoded_string).decode('unicode_escape')
    except:
        pass

    # Декодирование JSON
    try:
        decoded_string = json.loads(encoded_string)
    except:
        pass

    # Декодирование URL
    try:
        decoded_string = urllib.parse.unquote(encoded_string)
    except:
        pass

    # Декодирование Base64
    try:
        decoded_string = base64.b64decode(encoded_string).decode('utf-8')
    except:
        pass

    return decoded_string


import pandas as pd
import re


# Определение функции приведения к нижнему регистру
def lowercase_sql(query):
    return query.lower()


# Определение функции обобщения запроса
def generalize_sql(query):
    # Замена всех чисел на "0"
    generalized_query = re.sub(r'\d+', '0', query)
    return generalized_query


# Определение функции токенизации
def tokenize_sql(query):
    # Добавление пробелов перед специальными символами, такими как "<>", "!=", и т. д.
    query = re.sub(r'([<>!=])', r' \1 ', query)
    # Разделение запроса на токены
    tokens = query.split()
    return ' '.join(tokens)


# Обработка столбца "Query", создание нового столбца "Text"
df['Text'] = df['Query'].apply(decode_sql)  # Декодирование
df['Text'] = df['Text'].apply(lowercase_sql)  # Приведение к нижнему регистру
df['Text'] = df['Text'].apply(generalize_sql)  # Обобщение
df['Text'] = df['Text'].apply(tokenize_sql)  # Токенизация

print(df.head())

                                               Query  Label  \
0  SELECT * FROM universe WHERE fifty = 'better' ...      0   
1  SELECT AVG ( source ) FROM failed SELECT SUM (...      0   
2  1'   )    )     )   waitfor delay '0:0:5' and ...      1   
3  INSERT INTO seldom  ( tomorrow, nearly, scient...      0   
4                                 prado de la guzpea      2   

                                                Text  
0  select * from universe where fifty = 'better' ...  
1  select avg ( source ) from failed select sum (...  
2  0' ) ) ) waitfor delay '0:0:0' and ( ( ( 'clcf...  
3  insert into seldom ( tomorrow, nearly, scienti...  
4                                 prado de la guzpea  


In [22]:
# Разделение данных на обучающую и тестовую выборки в соотношении 80%/20%
train_df, test_df = train_test_split(df, test_size=0.20, random_state=50, shuffle=True)
train_df

,Query,Label,Text
3357,1' in boolean mode ) and 6969 = ( sele...,1,0' in boolean mode ) and 0 = ( select 0 from p...
9793,1' where 4149 = 4149 and 3202 = like ( '...,1,"0' where 0 = 0 and 0 = like ( 'abcdefg',upper ..."
6002,"SELECT prepare, worth+ ', ' + composed+ ' '...",0,"select prepare, worth+ ', ' + composed+ ' ' + ..."
5734,ceciley,2,ceciley
241,"1"" ) as sstd where 9629 = 9629 or updatex...",1,"0"" ) as sstd where 0 = 0 or updatexml ( 0,conc..."
...,...,...,...
8262,UPDATE stared SET canal = 'drove'WHERE pale =...,0,update stared set canal = 'drove'where pale = ...
6214,SELECT * FROM lower WHERE football BETWEEN 10...,0,select * from lower where football between 0 a...
8324,"-9119"" ) where 9237 = 9237 or 8571 = 85...",1,"-0"" ) where 0 = 0 or 0 = 0--"
6253,SELECT * FROM CustomersFETCH FIRST 3 ROWS ONLY...,0,select * from customersfetch first 0 rows only...


In [23]:
train_texts, train_labels = train_df['Text'].tolist(), train_df['Label'].tolist()
test_texts, test_labels = test_df['Text'].tolist(), test_df['Label'].tolist()

### BERT LSTM

In [24]:
# Определение класса пользовательского датасета
class CustomDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length):
        # Конструктор класса, инициализирующий атрибуты датасета
        self.texts = texts  # Список текстовых данных
        self.labels = labels  # Список меток
        self.tokenizer = tokenizer  # Токенизатор для предобработки текста
        self.max_length = max_length  # Максимальная длина последовательности

    def __len__(self):
        # Метод, возвращающий общее количество примеров в датасете
        return len(self.texts)

    def __getitem__(self, idx):
        # Метод, возвращающий примеры по индексу
        text = str(self.texts[idx])  # Получение текста по индексу
        label = self.labels[idx]  # Получение метки по индексу
        # Преобразование текста в токены с помощью токенизатора
        encoding = self.tokenizer(text, truncation=True, padding='max_length', max_length=self.max_length,
                                  return_tensors='pt')
        # Возвращение словаря с данными: токены входной последовательности, маска внимания и метка
        return {
            'input_ids': encoding['input_ids'].squeeze(),  # Токены входной последовательности
            'attention_mask': encoding['attention_mask'].squeeze(),  # Маска внимания
            'labels': torch.tensor(label, dtype=torch.long)  # Метка
        }

In [25]:
# Определение модели BERT+LSTM
class BertLSTMClassifier(nn.Module):
    def __init__(self, bert_model, hidden_size, output_size, num_layers, bidirectional=True):
        # Конструктор класса, инициализирующий атрибуты модели
        super(BertLSTMClassifier, self).__init__()
        self.bert_model = bert_model  # Модель BERT
        # LSTM слой, принимающий на вход скрытый размер BERT и возвращающий выход с заданным размером
        self.lstm = nn.LSTM(bert_model.config.hidden_size, hidden_size, num_layers, bidirectional=bidirectional,
                            batch_first=True)
        self.dropout = nn.Dropout(0.2)  # Dropout слой для регуляризации
        # Полносвязный слой для классификации с размером выхода равным output_size
        self.fc = nn.Linear(hidden_size * 2 if bidirectional else hidden_size, output_size)

    def forward(self, input_ids, attention_mask):
        # Проход прямого распространения через модель
        with torch.no_grad():
            outputs = self.bert_model(input_ids=input_ids, attention_mask=attention_mask)
        # Проход через LSTM слой
        lstm_output, _ = self.lstm(outputs.last_hidden_state)
        lstm_output = self.dropout(lstm_output)  # Применение dropout
        # Преобразование выхода LSTM в логиты с помощью полносвязного слоя
        logits = self.fc(lstm_output[:, -1, :])  # Взятие выхода с последнего временного шага
        return logits

In [26]:
# Определение модели BERT+TextCNN
class BertTextCNNClassifier(nn.Module):
    def __init__(self, bert_model, num_filters, filter_sizes, output_size):
        # Конструктор класса, инициализирующий атрибуты модели
        super(BertTextCNNClassifier, self).__init__()
        self.bert_model = bert_model  # Модель BERT
        self.num_filters = num_filters  # Количество фильтров
        self.filter_sizes = filter_sizes  # Размеры фильтров
        # Сверточные слои с одномерными ядрами, принимающие на вход скрытый размер BERT
        # и возвращающие выход с заданным количеством фильтров
        self.conv_layers = nn.ModuleList([
            nn.Conv1d(in_channels=bert_model.config.hidden_size, out_channels=num_filters, kernel_size=fs)
            for fs in filter_sizes
        ])
        self.dropout = nn.Dropout(0.2)  # Dropout слой для регуляризации
        # Полносвязный слой для классификации с размером выхода равным output_size
        self.fc = nn.Linear(num_filters * len(filter_sizes), output_size)

    def forward(self, input_ids, attention_mask):
        # Проход прямого распространения через модель
        with torch.no_grad():
            outputs = self.bert_model(input_ids=input_ids, attention_mask=attention_mask)

        # Транспонирование для соответствия формату входа Conv1d (batch_size, num_channels, seq_len)
        embedded = outputs.last_hidden_state.transpose(1, 2)

        # Применение сверточных слоев с разными размерами ядер и максимальным пулингом
        pooled_outputs = []
        for conv_layer in self.conv_layers:
            conv_out = nn.functional.relu(conv_layer(embedded))
            pooled_out, _ = torch.max(conv_out, dim=2)
            pooled_outputs.append(pooled_out)

        # Конкатенация и выравнивание сгруппированных выходов
        pooled_outputs = torch.cat(pooled_outputs, dim=1)
        pooled_outputs = self.dropout(pooled_outputs)

        # Пропуск выхода через полносвязный слой для классификации
        logits = self.fc(pooled_outputs)
        return logits

In [27]:
# Задание параметров обучения
batch_size = 64
max_length = 128
hidden_size = 128
num_layers = 1
# Количество классов в вашей задаче классификации
num_classes = 2
output_size = num_classes
bidirectional = False

# Загрузка модели BERT и токенизатора
bert_model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(bert_model_name)
bert_model = BertModel.from_pretrained(bert_model_name)

# Создание датасета и загрузчиков данных
train_dataset = CustomDataset(train_texts, train_labels, tokenizer, max_length)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = CustomDataset(test_texts, test_labels, tokenizer, max_length)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

In [33]:
# Инициализация модели TextCNN
num_filters = 100
filter_sizes = [2, 3, 4]
output_size = 3
model = BertTextCNNClassifier(bert_model, num_filters, filter_sizes, output_size)

# Определение функции потерь и оптимизатора
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=2e-5)

# Обучение модели
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.train()

num_epochs = 2  # Количество эпох обучения

# Обучение модели
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.train()

print("Start training")
for epoch in range(num_epochs):
    total_loss = 0
    correct_train = 0
    total_train = 0

    # Оборачиваем train_loader в tqdm для добавления индикатора прогресса
    with tqdm(train_loader, unit="batch") as t:
        for batch in t:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            optimizer.zero_grad()
            logits = model(input_ids, attention_mask)
            loss = criterion(logits, labels)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

            _, predicted = torch.max(logits.data, 1)
            total_train += labels.size(0)
            correct_train += (predicted == labels).sum().item()

            t.set_postfix({'loss': total_loss / (t.n + 1), 'accuracy': correct_train / total_train})

# Тестирование модели
model.eval()
with torch.no_grad():
    y_true = []
    y_pred = []

    # Оборачиваем test_loader в tqdm для добавления индикатора прогресса
    with tqdm(test_loader, unit="batch") as t:
        for batch in t:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            logits = model(input_ids, attention_mask)
            _, predicted = torch.max(logits.data, 1)

            y_true.extend(labels.cpu().numpy())
            y_pred.extend(predicted.cpu().numpy())

            t.set_postfix({})

Start training


100%|██████████| 32/32 [07:09<00:00, 13.42s/batch]


ValueError: Target is multiclass but average='binary'. Please choose another average setting, one of [None, 'micro', 'macro', 'weighted'].

In [35]:
# Вычисление метрик оценки
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, average='macro')
recall = recall_score(y_true, y_pred, average='macro')
f1 = f1_score(y_true, y_pred, average='macro')

print(f"Test Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")

Test Accuracy: 0.9825
Precision: 0.9823
Recall: 0.9834
F1-score: 0.9828


## Word2Vec LSTM

In [36]:
train_texts, train_labels = train_df['Text'].tolist(), train_df['Label'].tolist()
test_texts, test_labels = test_df['Text'].tolist(), test_df['Label'].tolist()

In [37]:
# Определение класса пользовательского датасета
class CustomDataset(Dataset):
    def __init__(self, texts, labels, word2vec_model, max_length):
        # Конструктор класса, инициализирующий атрибуты датасета
        self.texts = texts  # Список текстовых данных
        self.labels = labels  # Список меток
        self.word2vec_model = word2vec_model  # Модель Word2Vec
        self.word2vec_dim = word2vec_model.vector_size  # Размерность вектора Word2Vec
        self.max_length = max_length  # Максимальная длина последовательности

    def __len__(self):
        # Метод, возвращающий общее количество примеров в датасете
        return len(self.texts)

    def __getitem__(self, idx):
        # Метод, возвращающий примеры по индексу
        text = str(self.texts[idx])  # Получение текста по индексу
        label = self.labels[idx]  # Получение метки по индексу

        # Получение вектора Word2Vec для каждого слова
        words = text.split()
        embeddings = [self.get_word_embedding(word) for word in words]

        # Заполнение или обрезка векторов Word2Vec до одинаковой длины
        if len(embeddings) < self.max_length:
            embeddings.extend(
                [torch.zeros(self.word2vec_dim, dtype=torch.float32)] * (self.max_length - len(embeddings)))
        else:
            embeddings = embeddings[:self.max_length]

        return {
            'word2vec_embeddings': torch.stack(embeddings),  # Векторы Word2Vec
            'labels': torch.tensor(label, dtype=torch.long)  # Метка
        }

    def get_word_embedding(self, word):
        # Метод для получения вектора Word2Vec для слова
        if word in self.word2vec_model.wv:
            return torch.tensor(self.word2vec_model.wv[word], dtype=torch.float32)
        else:
            return torch.zeros(self.word2vec_dim, dtype=torch.float32)  # Вектор нулей, если слово отсутствует в модели

In [38]:
# Определение модели Word2Vec+LSTM
class Word2VecLSTMClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers, bidirectional=True):
        # Конструктор класса, инициализирующий атрибуты модели
        super(Word2VecLSTMClassifier, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, bidirectional=bidirectional, batch_first=True)
        self.dropout = nn.Dropout(0.2)  # Dropout слой для регуляризации
        # Полносвязный слой для классификации с размером выхода равным output_size
        self.fc = nn.Linear(hidden_size * 2 if bidirectional else hidden_size, output_size)

    def forward(self, word2vec_embeddings):
        # Проход прямого распространения через модель
        lstm_output, _ = self.lstm(word2vec_embeddings)
        lstm_output = self.dropout(lstm_output)  # Применение dropout
        # Преобразование выхода LSTM в логиты с помощью полносвязного слоя
        logits = self.fc(lstm_output[:, -1, :])  # Взятие выхода с последнего временного шага
        return logits

In [39]:
# Определение модели Word2Vec+TextCNN
class Word2VecTextCNNClassifier(nn.Module):
    def __init__(self, input_size, num_filters, filter_sizes, output_size):
        # Конструктор класса, инициализирующий атрибуты модели
        super(Word2VecTextCNNClassifier, self).__init__()
        self.num_filters = num_filters  # Количество фильтров
        self.filter_sizes = filter_sizes  # Размеры фильтров
        # Сверточные слои с одномерными ядрами, принимающие на вход размер вектора Word2Vec
        # и возвращающие выход с заданным количеством фильтров
        self.conv_layers = nn.ModuleList([
            nn.Conv1d(in_channels=input_size, out_channels=num_filters, kernel_size=fs)
            for fs in filter_sizes
        ])
        self.dropout = nn.Dropout(0.2)  # Dropout слой для регуляризации
        # Полносвязный слой для классификации с размером выхода равным output_size
        self.fc = nn.Linear(num_filters * len(filter_sizes), output_size)

    def forward(self, word2vec_embeddings):
        # Транспонирование для соответствия формату входа Conv1d (batch_size, num_channels, seq_len)
        embedded = word2vec_embeddings.transpose(1, 2)

        # Применение сверточных слоев с разными размерами ядер и максимальным пулингом
        pooled_outputs = []
        for conv_layer in self.conv_layers:
            conv_out = nn.functional.relu(conv_layer(embedded))
            pooled_out, _ = torch.max(conv_out, dim=2)
            pooled_outputs.append(pooled_out)

        # Конкатенация и выравнивание сгруппированных выходов
        pooled_outputs = torch.cat(pooled_outputs, dim=1)
        pooled_outputs = self.dropout(pooled_outputs)

        # Полносвязный слой для классификации
        logits = self.fc(pooled_outputs)
        return logits

In [40]:
# Обучение модели Word2Vec
sentences = [text.split() for text in train_texts + test_texts]
word2vec_model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)

batch_size = 64
max_length = 10

# Создание датасета и загрузчиков данных
train_dataset = CustomDataset(train_texts, train_labels, word2vec_model, max_length)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = CustomDataset(test_texts, test_labels, word2vec_model, max_length)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

In [ ]:
# Инициализация модели Word2Vec LSTM
# Задание параметров обучения
input_size = word2vec_model.vector_size
hidden_size = 128
num_layers = 1
output_size = 3
bidirectional = True
model = Word2VecLSTMClassifier(input_size, hidden_size, output_size, num_layers, bidirectional=bidirectional)
input_size = word2vec_model.vector_size  # Предполагая, что каждое вложение Word2Vec имеет размерность 100
num_filters = 100
filter_sizes = [2, 3, 4]
output_size = 3

In [42]:
# Инициализация модели Word2VecTextCNNClassifier
model = Word2VecTextCNNClassifier(input_size, num_filters, filter_sizes, output_size)

# Определение функции потерь и оптимизатора
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=2e-5)

# Обучение модели
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.train()

num_epochs = 2

# Обучение модели
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.train()

print("Start training")
for epoch in range(num_epochs):
    total_loss = 0
    correct_train = 0
    total_train = 0

    # Использование tqdm для отображения прогресса обучения
    with tqdm(train_loader, unit="batch") as t:
        for batch in t:
            word2vec_embeddings = batch['word2vec_embeddings'].to(device)
            labels = batch['labels'].to(device)

            optimizer.zero_grad()
            logits = model(word2vec_embeddings)
            loss = criterion(logits, labels)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

            _, predicted = torch.max(logits.data, 1)
            total_train += labels.size(0)
            correct_train += (predicted == labels).sum().item()

            t.set_postfix({'loss': total_loss / (t.n + 1), 'accuracy': correct_train / total_train})

# Тестирование модели
model.eval()
with torch.no_grad():
    y_true = []
    y_pred = []

    # Использование tqdm для отображения прогресса тестирования
    with tqdm(test_loader, unit="batch") as t:
        for batch in t:
            word2vec_embeddings = batch['word2vec_embeddings'].to(device)
            labels = batch['labels'].to(device)

            logits = model(word2vec_embeddings)
            _, predicted = torch.max(logits.data, 1)

            y_true.extend(labels.cpu().numpy())
            y_pred.extend(predicted.cpu().numpy())

            t.set_postfix({})

Start training


100%|██████████| 32/32 [00:00<00:00, 64.48batch/s]


In [43]:
# Вычисление метрик оценки
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, average='macro')
recall = recall_score(y_true, y_pred, average='macro')
f1 = f1_score(y_true, y_pred, average='macro')

print(f"Test Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")

Test Accuracy: 0.6960
Precision: 0.4863
Recall: 0.6306
F1-score: 0.5409


C:\Users\Admin\PycharmProjects\100DaysOfDataScience\venv\Lib\site-packages\sklearn\metrics\_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


## Попробуем больше эпох для Word2Vec

In [44]:
# Инициализация модели Word2VecTextCNNClassifier
model = Word2VecTextCNNClassifier(input_size, num_filters, filter_sizes, output_size)

# Определение функции потерь и оптимизатора
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=2e-5)

# Обучение модели
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.train()

num_epochs = 5

# Обучение модели
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.train()

print("Start training")
for epoch in range(num_epochs):
    total_loss = 0
    correct_train = 0
    total_train = 0

    # Использование tqdm для отображения прогресса обучения
    with tqdm(train_loader, unit="batch") as t:
        for batch in t:
            word2vec_embeddings = batch['word2vec_embeddings'].to(device)
            labels = batch['labels'].to(device)

            optimizer.zero_grad()
            logits = model(word2vec_embeddings)
            loss = criterion(logits, labels)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

            _, predicted = torch.max(logits.data, 1)
            total_train += labels.size(0)
            correct_train += (predicted == labels).sum().item()

            t.set_postfix({'loss': total_loss / (t.n + 1), 'accuracy': correct_train / total_train})

# Тестирование модели
model.eval()
with torch.no_grad():
    y_true = []
    y_pred = []

    # Использование tqdm для отображения прогресса тестирования
    with tqdm(test_loader, unit="batch") as t:
        for batch in t:
            word2vec_embeddings = batch['word2vec_embeddings'].to(device)
            labels = batch['labels'].to(device)

            logits = model(word2vec_embeddings)
            _, predicted = torch.max(logits.data, 1)

            y_true.extend(labels.cpu().numpy())
            y_pred.extend(predicted.cpu().numpy())

            t.set_postfix({})

Start training


100%|██████████| 32/32 [00:00<00:00, 68.46batch/s]


In [45]:
# Вычисление метрик оценки
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, average='macro')
recall = recall_score(y_true, y_pred, average='macro')
f1 = f1_score(y_true, y_pred, average='macro')

print(f"Test Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")

Test Accuracy: 0.9270
Precision: 0.9367
Recall: 0.9208
F1-score: 0.9257


In [46]:
# Инициализация модели Word2VecTextCNNClassifier
model = Word2VecTextCNNClassifier(input_size, num_filters, filter_sizes, output_size)

# Определение функции потерь и оптимизатора
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=2e-5)

# Обучение модели
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.train()

num_epochs = 10

# Обучение модели
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.train()

print("Start training")
for epoch in range(num_epochs):
    total_loss = 0
    correct_train = 0
    total_train = 0

    # Использование tqdm для отображения прогресса обучения
    with tqdm(train_loader, unit="batch") as t:
        for batch in t:
            word2vec_embeddings = batch['word2vec_embeddings'].to(device)
            labels = batch['labels'].to(device)

            optimizer.zero_grad()
            logits = model(word2vec_embeddings)
            loss = criterion(logits, labels)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

            _, predicted = torch.max(logits.data, 1)
            total_train += labels.size(0)
            correct_train += (predicted == labels).sum().item()

            t.set_postfix({'loss': total_loss / (t.n + 1), 'accuracy': correct_train / total_train})

# Тестирование модели
model.eval()
with torch.no_grad():
    y_true = []
    y_pred = []

    # Использование tqdm для отображения прогресса тестирования
    with tqdm(test_loader, unit="batch") as t:
        for batch in t:
            word2vec_embeddings = batch['word2vec_embeddings'].to(device)
            labels = batch['labels'].to(device)

            logits = model(word2vec_embeddings)
            _, predicted = torch.max(logits.data, 1)

            y_true.extend(labels.cpu().numpy())
            y_pred.extend(predicted.cpu().numpy())

            t.set_postfix({})

Start training


100%|██████████| 32/32 [00:00<00:00, 71.98batch/s]


In [47]:
# Вычисление метрик оценки
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, average='macro')
recall = recall_score(y_true, y_pred, average='macro')
f1 = f1_score(y_true, y_pred, average='macro')

print(f"Test Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")

Test Accuracy: 0.9435
Precision: 0.9483
Recall: 0.9411
F1-score: 0.9436
